In [ ]:
# pip install langchain qdrant_client openai streamlit

In [ ]:
### DO NOT RUN ALL ###

In [ ]:
### these are functions to run individually as needed to modify the Qdrant collections!! the py. files are to run the script ###

In [9]:
from dotenv import load_dotenv
import streamlit as st

In [10]:
# Imports
from langchain.vectorstores import Qdrant
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from qdrant_client import QdrantClient, models
import os
import qdrant_client


In [11]:
# create qdrant client
client = qdrant_client.QdrantClient(
        st.secrets["QDRANT_HOST"], 
        api_key=st.secrets["QDRANT_API_KEY"]
    )

In [13]:
# create collection

# define parameters for vectors
vectors_config = models.VectorParams(
    size=1536, # vector size for OpenAI
    distance=models.Distance.COSINE
)

# create collection
client.recreate_collection(
    collection_name=st.secrets["QDRANT_COLLECTION_NAME"],
    vectors_config=vectors_config,
)

True

In [14]:
# create vector store

embeddings = OpenAIEmbeddings()

# get collection name
vector_store = Qdrant(
    client=client, 
    collection_name=os.getenv("QDRANT_COLLECTION_NAME"), 
    embeddings=embeddings,
)

In [18]:
# add 'extracted text' to vector store

from langchain_text_splitters import RecursiveCharacterTextSplitter

def get_text_chunks(text):
    
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len,
        is_separator_regex=False,
    )
     
    chunks = text_splitter.split_text(text)
    return chunks

with open('combined_markdown_and_code.txt') as f:
    raw_text = f.read()
    
texts = get_text_chunks(raw_text)

metadatas = [{"name": raw_text} for _ in texts]
vector_store.add_texts(texts, metadatas=metadatas)

ResponseHandlingException: Server disconnected without sending a response.

In [15]:
from langchain_community.document_loaders import PyPDFLoader
# add 'extracted pdf' to vector store

from langchain_text_splitters import RecursiveCharacterTextSplitter

def get_pdf_chunks(pdf_filepath):
    loader = PyPDFLoader(pdf_filepath)
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
    )
     
    chunks = text_splitter.split_text(pdf_filepath)
    return chunks


slides_filepath = 'Slides'
for slide in os.listdir(slides_filepath):
    slide_path = os.path.join(slides_filepath, slide)
    texts = get_pdf_chunks(slide_path)
    metadatas = [{"name": slide} for _ in texts]
    vector_store.add_texts(texts, metadatas=metadatas)
    


In [16]:
# add transcipts to vector store

from langchain_text_splitters import RecursiveCharacterTextSplitter

def get_text_chunks(text_filepath):
    
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len,
        is_separator_regex=False,
    )
    chunks = text_splitter.split_text(text_filepath)
    return chunks


transcripts_filepath = 'Transcripts'
for transcript in os.listdir(transcripts_filepath):
    texts = get_text_chunks(transcript)
    metadatas = [{"name": transcript} for _ in texts]
    vector_store.add_texts(texts, metadatas=metadatas)
    


In [ ]:
# delete collection

client.delete_collection(collection_name=os.getenv("QDRANT_COLLECTION_NAME"))

In [ ]:
from langchain.schema import retriever

from langchain_community.llms import OpenAI
from langchain.chains import RetrievalQA

# add vector store to retrieval chain

retrievalQA = RetrievalQA.from_llm(
    llm=OpenAI(),
    retriever=vector_store.as_retriever()
    )

In [ ]:
# # conversational retrieval qa
# from langchain.chains import ConversationalRetrievalChain
# from langchain.chat_models import ChatOpenAI

# llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=1)
# crc = ConversationalRetrievalChain.from_llm(llm,retriever)

In [ ]:
query = "What topic are you knowledgable about?"

response = retrievalQA.run(query)

print(response)

In [ ]:
# playing with setting up prompt template
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate, FewShotPromptTemplate


def add_flair(crc_response):
    # define model
    llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=1)

    # Define a prefix that explains the prompt.
    prefix = """
    Here are examples between a human and AI. The human provides a word, and
    the AI provides a single sentence with easy to read words that mostly rhyme
    with the word the human provided. The sentence does not have to include the 
    original word. For example:
    """

    # Create examples.
    examples = [
        {
            "query": "rat",
            "answer": "The cat sat next to the bat."
        }, {
            "query": "frog",
            "answer": "A dog hops a log in the bog."
        }, {
            "query": "ten",
            "answer": "Ben sent ten hens to the glen."
        }
    ]

    # Define a format for the examples.
    example_format = """
    Human: {query}
    AI: {answer}
    """

    # Create a prompt template for the examples.
    example_template = PromptTemplate(
        input_variables=["query", "answer"],
        template=example_format
    )

    # Provide a suffix that includes the query.
    suffix = """
    Human: {query}
    AI: 
    """

    # Construct the few shot prompt template.
    prompt_template = FewShotPromptTemplate(
        examples=examples,
        example_prompt=example_template,
        input_variables=["query"],
        prefix=prefix,
        suffix=suffix,
        example_separator="\n\n"
    )

    chain = LLMChain(llm=llm, prompt=prompt_template, verbose=True)

    query = {"query": crc_response}

    result = chain.invoke(query)
    return result["text"]

In [ ]:
add_flair('Python is a high-level, interpreted programming language designed by Guido van Rossum. It was released in 1991 and has since become one of the most popular languages in use today, especially among web developers.')